In [270]:
from datetime import date
import holidays
import pandas as pd
import numpy as np
import pycountry

In [3]:
holiday_dates = []
holiday_names = []
for holiday in holidays.AO(years=[2019, 2021]).values():
    holiday_names.append(holiday)

for holiday in holidays.AO(years=[2019, 2021]).keys():
    holiday_dates.append(holiday)

In [4]:
d = {"Holidays": holiday_names, "Dates": holiday_dates}
holiday_df = pd.DataFrame(data=d)

In [314]:
from bs4 import BeautifulSoup
import requests
url = "https://pypi.org/project/holidays/"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
table = soup.find('table')
country_df = pd.read_html(str(table))[0]
country_df = country_df.dropna()
country_list = list(country_df["Code"])

In [322]:
hd = []
d = {"Holidays": holiday_names, "Dates": holiday_dates}
for country in country_list:
    for holiday in holidays.CountryHoliday(country, years=[2020]).items():
        hd.append({'Date' : holiday[0], "Name" : holiday[1], "Country" : country})

hd_df = pd.DataFrame(data = hd)

conversion_dict = {}
for c in pycountry.countries:
    conversion_dict[c.alpha_2] = c.name
    
hd_df = hd_df.replace({"Country": conversion_dict})
hd_df.reset_index(drop=True, inplace=True)

def date_converter(date):
    return date.strftime("%Y-%m-%d")
    
hd_df["Date"] = hd_df["Date"].map(date_converter)
hd_df

def remove_observed(holiday):
    return (not ("Observed" in holiday))

hd_df = hd_df[hd_df["Name"].str.contains("Observed") == False]
hd_df.reset_index(drop=True, inplace=True)
#print(hd_df)

hd_df = hd_df.groupby(["Name", "Date"])["Country"].apply(list).reset_index(name='countries')
hd_df.columns = ["name", "date", "countries"]

In [321]:
json_hd = hd_df.to_json(orient = "records") 
with open('/Users/bjornholst/Desktop/holiday_json', 'w') as file:
    file.write(json_hd)
#to_json('temp.json', orient='records', lines=True)